In [1]:
#!pip install fsspec
#!pip install gcsfs 
 
 


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
import scipy
import seaborn as sns
import math
from scipy import stats

In [3]:
df = pd.read_csv('gs://capstoragebucket2020/All_joined_v11.csv')

In [4]:
df2=df.copy()


df2.head(10)

,latitude,longitude,quarter,year,sum_etage_hors_sol,sum_nombre_logement,nombre_logement_per_population,avg_annee_construction,sum_superficie_terrain,area,...,superficie_batiment_per_population,incendie_count_last_100,incendie_count_last_300,alarm_incendie_count_last_100,total_crimes_last_100,vols_count_last_100,mefait_count_last_100,vol_de_vehicule_count_last_100,introduction_count_last_100,infractions_entrainant_count_last_100
0,45.40,-73.96,2,2017,0,0,0.000000,1900.00,2732,1.57174,...,2.360795,0,0,1,0,0,0,0,0,0
1,45.40,-73.96,3,2019,0,0,0.000000,1900.00,2732,1.57174,...,2.360795,0,0,0,0,0,0,0,0,0
2,45.41,-73.95,1,2015,728,730,2.324841,1949.98,253537,0.21093,...,257.929936,0,0,0,0,0,0,0,0,0
3,45.41,-73.95,2,2015,728,730,2.324841,1949.98,253537,0.21093,...,257.929936,0,0,2,5,1,1,1,1,0
4,45.41,-73.95,3,2015,728,730,2.324841,1949.98,253537,0.21093,...,257.929936,0,0,3,7,1,2,1,1,0
5,45.41,-73.95,4,2015,728,730,2.324841,1949.98,253537,0.21093,...,257.929936,0,0,1,7,0,1,2,2,0
6,45.41,-73.95,1,2016,728,730,2.324841,1949.98,253537,0.21093,...,257.929936,1,1,0,3,0,2,0,1,0
7,45.41,-73.95,2,2016,728,730,2.324841,1949.98,253537,0.21093,...,257.929936,1,1,4,2,0,2,0,0,0
8,45.41,-73.95,3,2016,728,730,2.324841,1949.98,253537,0.21093,...,257.929936,0,1,2,4,0,1,0,2,0
9,45.41,-73.95,4,2016,728,730,2.324841,1949.98,253537,0.21093,...,257.929936,0,1,0,3,1,0,0,2,0


In [5]:
!pip install geopy 
import geopy.distance

def dist(x,y):
  coords_1 = (45.504654, -73.56546)
  coords_2 = (x, y)
  return geopy.distance.distance(coords_1, coords_2).km

df2['distance']=0

for i in range(0,len(df2)-1):
  df2['distance'][i]=dist(df2['latitude'][i], df2['longitude'][i])

<ipython-input-5-325eb8ccd029>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['distance'][i]=dist(df2['latitude'][i], df2['longitude'][i])


In [6]:
df2['incendie_count'].value_counts()

0     7633
1     2644
2      961
3      362
4      151
5       55
6       19
7        9
9        4
8        4
10       1
Name: incendie_count, dtype: int64

In [7]:
df2['fire'] = df2['incendie_count']
df2['fire'][df2['incendie_count']==0] = 0
df2['fire'][(df2['incendie_count']==1)] = 1
df2['fire'][df2['incendie_count']>=2] = 2

<ipython-input-7-111c3b4d9195>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['fire'][df2['incendie_count']==0] = 0
<ipython-input-7-111c3b4d9195>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['fire'][(df2['incendie_count']==1)] = 1
<ipython-input-7-111c3b4d9195>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['fire'][df2['incendie_count']>=2] = 2


In [8]:
df2['fire'].value_counts()

0    7633
1    2644
2    1566
Name: fire, dtype: int64

In [9]:
df2 = df2.drop("latitude", axis=1)
df2 = df2.drop("longitude", axis=1)     
df2 = df2.drop("incendie_count", axis=1)
df3 = df2.drop("year",axis=1)
df3 = df3.drop("quarter",axis=1)

In [10]:
corr_matrix = df2.corr()
corr_matrix["fire"].sort_values(ascending=False)

fire                                     1.000000
incendie_count_last_100                  0.505725
sum_nombre_logement                      0.503540
incendie_count_last_300                  0.489631
sum_etage_hors_sol                       0.445073
total_crimes_last_100                    0.432339
introduction_count_last_100              0.404394
mefait_count_last_100                    0.403223
alarm_incendie_count_last_100            0.397760
vols_count_last_100                      0.321872
vol_de_vehicule_count_last_100           0.218539
households_per_population                0.202810
caserne_count                            0.163537
dwellings_per_population                 0.153500
sum_superficie_batiment                  0.133144
sum_superficie_terrain                   0.085039
dwellings                                0.067331
infractions_entrainant_count_last_100    0.063383
households                               0.040217
nombre_logement_per_population           0.025062


In [11]:
X_raw=df3.drop("fire", axis=1)

In [12]:
from sklearn import preprocessing


# standardize X and y before fitting multiple linear model
X = pd.DataFrame(preprocessing.scale(X_raw))
y = df3['fire'].copy()

In [13]:
X

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,-1.265550,-1.029609,-0.095593,-4.048701,-2.432395,-0.022705,-0.120996,-0.753436,-0.630050,0.055159,...,-0.634121,-0.759640,-0.655738,-0.790765,-0.451506,-0.691828,-0.815238,-0.644038,-0.111488,2.857580
1,-1.265550,-1.029609,-0.095593,-4.048701,-2.432395,-0.022705,-0.120996,-0.753436,-0.630050,0.055159,...,-0.634121,-0.759640,-0.744615,-0.790765,-0.451506,-0.691828,-0.815238,-0.644038,-0.111488,2.857580
2,-0.620745,-0.626044,-0.084744,-1.180362,-0.977545,-0.438656,-0.148880,-0.829859,-1.246526,-2.005104,...,-0.634121,-0.759640,-0.744615,-0.790765,-0.451506,-0.691828,-0.815238,-0.644038,-0.111488,2.714514
3,-0.620745,-0.626044,-0.084744,-1.180362,-0.977545,-0.438656,-0.148880,-0.829859,-1.246526,-2.005104,...,-0.634121,-0.759640,-0.566862,-0.544342,0.053041,-0.500908,-0.456401,-0.516917,-0.111488,2.714514
4,-0.620745,-0.626044,-0.084744,-1.180362,-0.977545,-0.438656,-0.148880,-0.829859,-1.246526,-2.005104,...,-0.634121,-0.759640,-0.477986,-0.445772,0.053041,-0.309989,-0.456401,-0.516917,-0.111488,2.714514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11838,-0.840404,-0.796868,-0.092602,0.294555,-1.443760,-0.405018,-0.150227,-0.140038,-0.196527,-0.169464,...,1.047073,0.084975,-0.744615,-0.642911,-0.451506,-0.500908,-0.815238,-0.516917,-0.111488,1.426923
11839,-0.840404,-0.796868,-0.092602,0.294555,-1.443760,-0.405018,-0.150227,-0.140038,-0.196527,-0.169464,...,-0.634121,0.084975,-0.744615,-0.495057,-0.451506,-0.691828,-0.097564,-0.389797,-0.111488,1.426923
11840,-0.840404,-0.796868,-0.092602,0.294555,-1.443760,-0.405018,-0.150227,-0.140038,-0.196527,-0.169464,...,-0.634121,0.084975,-0.744615,-0.692196,-0.451506,-0.691828,-0.815238,-0.516917,-0.111488,1.426923
11841,-0.840404,-0.796868,-0.092602,0.294555,-1.443760,-0.405018,-0.150227,-0.140038,-0.196527,-0.169464,...,-0.634121,-0.759640,-0.389110,-0.790765,-0.451506,-0.691828,-0.815238,-0.644038,-0.111488,1.426923


In [29]:
X_whole = X[df2['year'] < 2019]
y_whole = y[df2['year'] < 2019]
X_valid = X[(df2['year']==2019) & (df2['quarter']==1)]
y_valid = y[(df2['year']==2019) & (df2['quarter']==1)]
X_valid2 = X[(df2['year']==2019) & (df2['quarter']==2)]
y_valid2 = y[(df2['year']==2019) & (df2['quarter']==2)]

X_test1= X[(df2['year']==2019) & (df2['quarter']==3)]
y_test1 = y[(df2['year']==2019) & (df2['quarter']==3)]

X_test2 = X[(df2['year']==2019) & (df2['quarter']==4)]
y_test2 = y[(df2['year']==2019) & (df2['quarter']==4)]

In [30]:
len(X_valid),len(y_valid),len(X_whole),len(y_whole),len(X_test1),len(y_test1)

(494, 494, 7999, 7999, 502, 502)

In [18]:
#!pip install xgboost 

In [16]:
from sklearn.model_selection import KFold
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score,cohen_kappa_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

k = 5
kf = KFold(n_splits = k, random_state=0, shuffle=True)

accuracy_xgboost0 = 0
accuracy_randomforest0 = 0
accuracy_svm0 = 0
accuracy_knn0 = 0

kappa_xgboost0 = 0
kappa_randomforest0 = 0
kappa_svm0 = 0
kappa_knn0 = 0


param_xgboost = {}
param_xgboost['objective'] = 'multi:softmax'
param_xgboost['num_class'] = len(np.unique(y_whole)) + 1 
param_xgboost['max_depth'] = 5
param_xgboost['learning_rate'] = 0.2
param_xgboost['gamma'] = 0

param_randomforest = {}
param_randomforest['n_estimators'] = 200
param_randomforest['max_depth'] = 10

param_svm = {}
param_svm['probability'] = True
param_svm['class_weight'] = 'balanced'
param_svm['C'] = 0.01
param_svm['gamma'] = 'scale'
param_svm['kernel'] = 'rbf'

param_knn = {}
param_knn['n_neighbors'] = 8
param_knn['leaf_size'] = 15

for train_index, test_index in kf.split(y_whole):
      X_train, X_test = X_whole.iloc[train_index], X_whole.iloc[test_index]
      y_train, y_test = y_whole.iloc[train_index], y_whole.iloc[test_index]
      
      xgboost = XGBClassifier(**param_xgboost)
      xgboost.fit(X_train,y_train)
      y_pred_xgboost = xgboost.predict(X_test)
      confusion_matrix_xgboost = confusion_matrix(y_test, y_pred_xgboost)
      accuracy_xgboost = np.sum(np.diagonal(confusion_matrix_xgboost))/np.sum(confusion_matrix_xgboost)
      accuracy_xgboost0 = accuracy_xgboost0 + accuracy_xgboost
      kappa_xgboost = cohen_kappa_score(y_test, y_pred_xgboost)
      kappa_xgboost0 = kappa_xgboost0 + kappa_xgboost
      print("xgboost,"+str(accuracy_xgboost) + "," +str(kappa_xgboost))

      randomforest = RandomForestClassifier(**param_randomforest)
      randomforest.fit(X_train,y_train)
      y_pred_randomforest = randomforest.predict(X_test)
      confusion_matrix_randomforest = confusion_matrix(y_test, y_pred_randomforest)
      accuracy_randomforest = np.sum(np.triu(confusion_matrix_randomforest))/np.sum(confusion_matrix_randomforest) 
      accuracy_randomforest0 = accuracy_randomforest0 + accuracy_randomforest
      kappa_randomforest = cohen_kappa_score(y_test, y_pred_randomforest)
      kappa_randomforest0 = kappa_randomforest0 + kappa_randomforest
      print("RF,"+str(accuracy_randomforest) + "," +str(kappa_randomforest))

      svm = SVC(**param_svm)
      svm.fit(X_train,y_train)
      y_pred_svm = svm.predict(X_test)
      confusion_matrix_svm = confusion_matrix(y_test, y_pred_svm)
      accuracy_svm = np.sum(np.triu(confusion_matrix_svm))/np.sum(confusion_matrix_svm)
      accuracy_svm0 = accuracy_svm0 + accuracy_svm
      kappa_svm= cohen_kappa_score(y_test, y_pred_svm)
      kappa_svm0 = kappa_svm0 + kappa_svm
      print("SVM,"+str(accuracy_svm) + "," +str(kappa_svm))
      

      knn = KNeighborsClassifier(**param_knn)
      knn.fit(X_train,y_train)
      y_pred_knn = knn.predict(X_test)
      confusion_matrix_knn = confusion_matrix(y_test, y_pred_knn)
      accuracy_knn = np.sum(np.triu(confusion_matrix_knn))/np.sum(confusion_matrix_knn)
      accuracy_knn0 = accuracy_knn0 + accuracy_knn
      kappa_knn= cohen_kappa_score(y_test, y_pred_knn)
      kappa_knn0 = kappa_knn0 + kappa_knn      
      print("KNN,"+str(accuracy_knn) + "," +str(kappa_knn))

accuracy_xgboost_mean= accuracy_xgboost0/k
accuracy_randomforest_mean= accuracy_randomforest0/k
accuracy_svm_mean= accuracy_svm0/k
accuracy_knn_mean= accuracy_knn0/k

kappa_xgboost_mean = kappa_xgboost0/k
kappa_randomforest_mean = kappa_randomforest0/k
kappa_svm_mean = kappa_svm0/k
kappa_knn_mean = kappa_knn0/k

print(accuracy_xgboost_mean,kappa_xgboost_mean)
print(accuracy_randomforest_mean,kappa_randomforest_mean)
print(accuracy_svm_mean,kappa_svm_mean)
print(accuracy_knn_mean,kappa_knn_mean)

xgboost,0.660625,0.25797687668733815
RF,0.76625,0.2700319273126205
SVM,0.86875,0.281834712202348
KNN,0.755,0.2608641157406578
xgboost,0.67875,0.31287671049751264
RF,0.78,0.2998678535440703
SVM,0.891875,0.2907552592872469
KNN,0.761875,0.25126163478088237
xgboost,0.654375,0.270541174685932
RF,0.7525,0.2658257106534887
SVM,0.870625,0.2780141634740071
KNN,0.736875,0.22710390371290845
xgboost,0.64125,0.23779488349897293
RF,0.731875,0.22736329992859639
SVM,0.86375,0.2903787226682406
KNN,0.726875,0.20280439608034206
xgboost,0.6497811131957474,0.2766605408243634
RF,0.7535959974984365,0.27787538856602245
SVM,0.8724202626641651,0.26562201677397257
KNN,0.7310819262038775,0.24724741668982586
0.6569562226391494 0.2711700372388238
0.7568441994996873 0.26819283600095967
0.873484052532833 0.28132097488116303
0.7423413852407755 0.23785629340092332


In [17]:
from sklearn.ensemble import VotingClassifier

xgb = XGBClassifier(**param_xgboost)
knn = KNeighborsClassifier(**param_knn)
svm = SVC(**param_svm)
randomforest = RandomForestClassifier(**param_randomforest)

svm2 = SVC(C=0.01, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.15,
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.000001, verbose=False)


voting_clf = VotingClassifier(
    estimators=[('knn', knn), ('svm', svm), ('xgb', xgboost), ('RF', randomforest), ('svm2', svm2)],
    voting='hard')

In [18]:
voting_clf.fit(X_whole, y_whole)

VotingClassifier(estimators=[('knn',
                              KNeighborsClassifier(leaf_size=15,
                                                   n_neighbors=8)),
                             ('svm',
                              SVC(C=0.01, class_weight='balanced',
                                  probability=True)),
                             ('xgb',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                                            gpu_id=-1, importance_type='gain',
                                            interaction_constraints='',
                                            learning_rate=0.2, max_de...
                                            n_estimators=100, n_jobs=0,
                                            num_class=4, nu

In [19]:
from sklearn.metrics import accuracy_score

#for clf in (knn, svm, xgboost,randomforest, svm2, voting_clf):
for clf in (randomforest, svm, svm2, voting_clf):
    clf.fit(X_whole, y_whole)
    y_pred = clf.predict(X_valid)
    print(clf.__class__.__name__, accuracy_score(y_valid, y_pred))

RandomForestClassifier 0.6862348178137652
SVC 0.5668016194331984
SVC 0.46963562753036436
VotingClassifier 0.6740890688259109


In [20]:
from sklearn.metrics import confusion_matrix
y_valid_pred_voting = voting_clf.predict(X_valid)
confusion_matrix_valid = confusion_matrix(y_valid, y_valid_pred_voting)
print(confusion_matrix_valid) 

[[283  18  15]
 [ 79  12  18]
 [ 27   4  38]]


In [21]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import KFold
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score,cohen_kappa_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

accuracy_xgboost0 = 0
accuracy_randomforest0 = 0
accuracy_svm0 = 0
accuracy_knn0 = 0

kappa_xgboost0 = 0
kappa_randomforest0 = 0
kappa_svm0 = 0
kappa_knn0 = 0


param_xgboost = {}
param_xgboost['objective'] = 'multi:softmax'
param_xgboost['num_class'] = len(np.unique(y_whole)) + 1 
param_xgboost['max_depth'] = 5
param_xgboost['learning_rate'] = 0.2
param_xgboost['gamma'] = 0


param_randomforest = {}
param_randomforest['n_estimators'] = 200
param_randomforest['max_depth'] = 10

param_svm = {}
param_svm['probability'] = True
param_svm['class_weight'] = 'balanced'
param_svm['C'] = 0.01
param_svm['gamma'] = 'scale'
param_svm['kernel'] = 'rbf'

param_knn = {}
param_knn['n_neighbors'] = 8
param_knn['leaf_size'] = 15

xgb = XGBClassifier(**param_xgboost)
knn = KNeighborsClassifier(**param_knn)
svm = SVC(**param_svm)
randomforest = RandomForestClassifier(**param_randomforest)

svm2 = SVC(C=0.01, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.15,
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.000001, verbose=False)


svm3 =  SVC(C=0.01, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.12,
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
 

#voting_clf = VotingClassifier( estimators=[('knn', knn), ('svm', svm), ('xgb', xgboost), ('RF', randomforest)],voting='soft')    
voting_clf = VotingClassifier( estimators=[('knn', knn), ('svm', svm), ('svm2', svm2),('svm3', svm3),('xgb', xgboost), ('RF', randomforest)],voting='soft')    
voting_clf_1 = VotingClassifier( estimators=[('knn', knn), ('svm', svm),('xgb', xgboost), ('RF', randomforest)],voting='soft')    
voting_clf_2 = VotingClassifier( estimators=[('knn', knn), ('svm2', svm2),('xgb', xgboost), ('RF', randomforest)],voting='soft')    
voting_clf_3 = VotingClassifier( estimators=[('knn', knn), ('svm3', svm3),('xgb', xgboost), ('RF', randomforest)],voting='soft')    

In [42]:
voting_clf.fit(X_whole, y_whole)
voting_clf_1.fit(X_whole, y_whole)
voting_clf_2.fit(X_whole, y_whole)
voting_clf_3.fit(X_whole, y_whole)

VotingClassifier(estimators=[('knn',
                              KNeighborsClassifier(leaf_size=15,
                                                   n_neighbors=8)),
                             ('svm3',
                              SVC(C=0.01, class_weight='balanced', gamma=0.12,
                                  probability=True)),
                             ('xgb',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                                            gpu_id=-1, importance_type='gain',
                                            interaction_constraints='',
                                            learning_rat...
                                            max_depth=5, min_child_weight=1,
                                            missing=na

In [44]:
from sklearn.metrics import confusion_matrix
y_valid_pred_voting = voting_clf_3.predict(X_valid)
confusion_matrix_valid = confusion_matrix(y_valid, y_valid_pred_voting)
print(confusion_matrix_valid) 

[[293  13  10]
 [ 88  10  11]
 [ 25   8  36]]


In [22]:
##  1st quarter

from sklearn.metrics import accuracy_score

#for clf in (knn, svm, xgboost,randomforest, svm2, voting_clf):
for clf in (knn, svm,svm2,svm3, xgboost,randomforest, voting_clf,voting_clf_1,voting_clf_2,voting_clf_3):
    clf.fit(X_whole, y_whole)
    y_pred = clf.predict(X_valid)
    print(clf.__class__.__name__, accuracy_score(y_valid, y_pred))
    print("kappa", cohen_kappa_score(y_valid, y_pred))    
    print(confusion_matrix(y_valid, y_pred))

KNeighborsClassifier 0.6680161943319838
kappa 0.2707371301522147
[[281  26   9]
 [ 81  20   8]
 [ 29  11  29]]
SVC 0.5668016194331984
kappa 0.2608254847258057
[[206  60  50]
 [ 43  30  36]
 [  4  21  44]]
SVC 0.46963562753036436
kappa 0.20889948351211762
[[146  46 124]
 [ 26  26  57]
 [  3   6  60]]
SVC 0.5101214574898786
kappa 0.2436112394100638
[[164  47 105]
 [ 30  28  51]
 [  3   6  60]]
XGBClassifier 0.6842105263157895
kappa 0.3084647206094815
[[288  14  14]
 [ 78  15  16]
 [ 27   7  35]]
RandomForestClassifier 0.6740890688259109
kappa 0.27584448693435315
[[292  12  12]
 [ 83   9  17]
 [ 25  12  32]]
VotingClassifier 0.680161943319838
kappa 0.28664911895883594
[[290  16  10]
 [ 86   9  14]
 [ 26   6  37]]
VotingClassifier 0.6902834008097166
kappa 0.30740683044836836
[[293  14   9]
 [ 85  11  13]
 [ 25   7  37]]
VotingClassifier 0.6842105263157895
kappa 0.29542129900526626
[[292  14  10]
 [ 87   9  13]
 [ 23   9  37]]
VotingClassifier 0.6862348178137652
kappa 0.29968171508012
[[291

In [24]:
##   2nd quarter

from sklearn.metrics import accuracy_score

#for clf in (knn, svm, xgboost,randomforest, svm2, voting_clf):
for clf in (knn, svm,svm2,svm3, xgboost,randomforest, voting_clf,voting_clf_1,voting_clf_2,voting_clf_3):
    clf.fit(X_whole, y_whole)
    ##clf.fit(X_valid, y_valid)
    y_pred2 = clf.predict(X_valid2)
    print(clf.__class__.__name__, accuracy_score(y_valid2, y_pred2))
    print("kappa", cohen_kappa_score(y_valid2, y_pred2))    
    print(confusion_matrix(y_valid2, y_pred2))

KNeighborsClassifier 0.6354378818737271
kappa 0.21136176016654107
[[277  23   7]
 [ 80  17  15]
 [ 34  20  18]]
SVC 0.5967413441955194
kappa 0.31516846413401056
[[210  48  49]
 [ 39  35  38]
 [  5  19  48]]
SVC 0.5071283095723014
kappa 0.26808505395949345
[[152  45 110]
 [ 17  31  64]
 [  2   4  66]]
SVC 0.5437881873727087
kappa 0.29776080807564864
[[171  52  84]
 [ 21  31  60]
 [  2   5  65]]
XGBClassifier 0.6435845213849287
kappa 0.2544080385963694
[[272  27   8]
 [ 81  13  18]
 [ 22  19  31]]
RandomForestClassifier 0.6619144602851323
kappa 0.27099209330614293
[[280  19   8]
 [ 82  17  13]
 [ 29  15  28]]
VotingClassifier 0.6619144602851323
kappa 0.271057291573506
[[283  15   9]
 [ 84  10  18]
 [ 26  14  32]]
VotingClassifier 0.6659877800407332
kappa 0.2843201734895213
[[280  18   9]
 [ 83  14  15]
 [ 26  13  33]]
VotingClassifier 0.6639511201629328
kappa 0.2771938902965633
[[280  17  10]
 [ 84  14  14]
 [ 27  13  32]]
VotingClassifier 0.6639511201629328
kappa 0.2802121648275494
[[28

In [39]:
y_pred_baseline1=[0 for x in y_test1]
print(confusion_matrix(y_test1, y_pred_baseline1))
print(accuracy_score(y_test1, y_pred_baseline1))




[[322   0   0]
 [111   0   0]
 [ 69   0   0]]
0.6414342629482072


In [40]:
y_pred_baseline1=[2 for x in y_test1]
print(confusion_matrix(y_test1, y_pred_baseline1))
print(accuracy_score(y_test1, y_pred_baseline1))



[[  0   0 322]
 [  0   0 111]
 [  0   0  69]]
0.13745019920318724


In [31]:
## 3rd quarter

from sklearn.metrics import accuracy_score

#for clf in (knn, svm, xgboost,randomforest, svm2, voting_clf):
for clf in (knn, svm,svm2,svm3, xgboost,randomforest, voting_clf,voting_clf_1,voting_clf_2,voting_clf_3):
    clf.fit(X_whole, y_whole)
    y_pred_test1 = clf.predict(X_test1)
    print(clf.__class__.__name__, accuracy_score(y_test1, y_pred_test1))
    print("kappa", cohen_kappa_score(y_test1, y_pred_test1))    
    print(confusion_matrix(y_test1, y_pred_test1))

KNeighborsClassifier 0.6713147410358565
kappa 0.282341422840656
[[291  21  10]
 [ 82  18  11]
 [ 20  21  28]]
SVC 0.5896414342629482
kappa 0.3015581415767826
[[212  53  57]
 [ 43  33  35]
 [  2  16  51]]
SVC 0.49203187250996017
kappa 0.2326505655762764
[[161  42 119]
 [ 26  26  59]
 [  0   9  60]]
SVC 0.5278884462151394
kappa 0.26508450286617913
[[178  45  99]
 [ 27  27  57]
 [  1   8  60]]
XGBClassifier 0.6812749003984063
kappa 0.29894999607230455
[[295  16  11]
 [ 84  13  14]
 [ 20  15  34]]
RandomForestClassifier 0.6892430278884463
kappa 0.31445979305635796
[[297  12  13]
 [ 83  17  11]
 [ 20  17  32]]
VotingClassifier 0.6812749003984063
kappa 0.2932190563348057
[[297  12  13]
 [ 87  10  14]
 [ 20  14  35]]
VotingClassifier 0.6852589641434262
kappa 0.30847363052215837
[[295  13  14]
 [ 84  14  13]
 [ 20  14  35]]
VotingClassifier 0.6812749003984063
kappa 0.29216641844315383
[[296  12  14]
 [ 86  11  14]
 [ 23  11  35]]
VotingClassifier 0.6772908366533864
kappa 0.28860974308283105
[[

In [33]:
## 4th quarter

from sklearn.metrics import accuracy_score

#for clf in (knn, svm, xgboost,randomforest, svm2, voting_clf):
for clf in (knn, svm,svm2,svm3, xgboost,randomforest, voting_clf,voting_clf_1,voting_clf_2,voting_clf_3):
    clf.fit(X_whole, y_whole)
    y_pred_test2 = clf.predict(X_test2)
    print(clf.__class__.__name__, accuracy_score(y_test2, y_pred_test2))
    print("kappa", cohen_kappa_score(y_test2, y_pred_test2))    
    print(confusion_matrix(y_test2, y_pred_test2))

KNeighborsClassifier 0.6391129032258065
kappa 0.17955162917921896
[[287  23   7]
 [ 90  16  13]
 [ 31  15  14]]
SVC 0.5463709677419355
kappa 0.22939352718183126
[[202  54  61]
 [ 44  32  43]
 [  9  14  37]]
SVC 0.4415322580645161
kappa 0.17510521923425604
[[149  46 122]
 [ 26  17  76]
 [  3   4  53]]
SVC 0.4778225806451613
kappa 0.2016257838378691
[[163  52 102]
 [ 30  23  66]
 [  4   5  51]]
XGBClassifier 0.6532258064516129
kappa 0.2573751512460938
[[277  29  11]
 [ 82  21  16]
 [ 21  13  26]]
RandomForestClassifier 0.6592741935483871
kappa 0.25597578619422534
[[283  19  15]
 [ 83  23  13]
 [ 24  15  21]]
VotingClassifier 0.655241935483871
kappa 0.2361465097219846
[[287  15  15]
 [ 87  15  17]
 [ 26  11  23]]
VotingClassifier 0.6512096774193549
kappa 0.2351341955841586
[[283  20  14]
 [ 86  18  15]
 [ 24  14  22]]
VotingClassifier 0.6491935483870968
kappa 0.2303082191780822
[[283  21  13]
 [ 85  18  16]
 [ 25  14  21]]
VotingClassifier 0.6491935483870968
kappa 0.23111736721130371
[[28

In [23]:
from sklearn.metrics import accuracy_score

#for clf in (knn, svm, xgboost,randomforest, svm2, voting_clf):
for clf in (knn, svm,svm2,svm3, xgboost,randomforest, voting_clf,voting_clf_1,voting_clf_2,voting_clf_3):
    clf.fit(X_whole, y_whole)
    y_pred = clf.predict(X_valid)
    print(clf.__class__.__name__, accuracy_score(y_valid, y_pred))
    print("kappa", cohen_kappa_score(y_valid, y_pred))
    print(confusion_matrix(y_valid, y_pred))

KNeighborsClassifier 0.6680161943319838
kappa 0.2707371301522147
[[281  26   9]
 [ 81  20   8]
 [ 29  11  29]]
SVC 0.5668016194331984
kappa 0.2608254847258057
[[206  60  50]
 [ 43  30  36]
 [  4  21  44]]
SVC 0.46963562753036436
kappa 0.20889948351211762
[[146  46 124]
 [ 26  26  57]
 [  3   6  60]]
SVC 0.5101214574898786
kappa 0.2436112394100638
[[164  47 105]
 [ 30  28  51]
 [  3   6  60]]
XGBClassifier 0.6842105263157895
kappa 0.3084647206094815
[[288  14  14]
 [ 78  15  16]
 [ 27   7  35]]
RandomForestClassifier 0.6821862348178138
kappa 0.29250248579221516
[[292  13  11]
 [ 84  10  15]
 [ 25   9  35]]
VotingClassifier 0.6862348178137652
kappa 0.2948576269937746
[[293  13  10]
 [ 86   9  14]
 [ 27   5  37]]
VotingClassifier 0.6902834008097166
kappa 0.3060905969409303
[[293  14   9]
 [ 85  11  13]
 [ 26   6  37]]
VotingClassifier 0.6842105263157895
kappa 0.2908764665286404
[[292  14  10]
 [ 87  10  12]
 [ 26   7  36]]
VotingClassifier 0.6842105263157895
kappa 0.29330851268695723
[[29

In [27]:
knn = KNeighborsClassifier(**param_knn)
knn.fit(X_whole, y_whole)
y_valid_pred_knn = knn.predict(X_valid)
from sklearn.metrics import confusion_matrix
confusion_matrix_valid = confusion_matrix(y_valid, y_valid_pred_knn)
print(confusion_matrix_valid) 

[[281  26   9]
 [ 81  20   8]
 [ 29  11  29]]


In [28]:
xgb = XGBClassifier(**param_xgboost)
xgb.fit(X_whole, y_whole)
y_valid_pred_xgb = xgb.predict(X_valid)
from sklearn.metrics import confusion_matrix
confusion_matrix_valid = confusion_matrix(y_valid, y_valid_pred_xgb)
print(confusion_matrix_valid) 

[[288  14  14]
 [ 78  15  16]
 [ 27   7  35]]


In [29]:
from sklearn.svm import SVC

final = SVC(**param_svm)
final.fit(X_whole, y_whole)
y_valid_pred_svc1 = final.predict(X_valid)
from sklearn.metrics import confusion_matrix
confusion_matrix_valid = confusion_matrix(y_valid, y_valid_pred_svc1)
print(confusion_matrix_valid) 

[[206  60  50]
 [ 43  30  36]
 [  4  21  44]]


In [30]:
final = SVC(C=0.01, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.11,
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
final.fit(X_whole, y_whole)
final.fit(X_whole, y_whole)
y_valid_pred_svc2 = final.predict(X_valid)
from sklearn.metrics import confusion_matrix
confusion_matrix_valid = confusion_matrix(y_valid, y_valid_pred_svc2)
print(confusion_matrix_valid) 

[[168  53  95]
 [ 33  28  48]
 [  3   8  58]]


In [31]:
### poly degree 3
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import LinearSVC

polynomial_svm_clf = Pipeline([
        ("poly_features", PolynomialFeatures(degree=3)),
        ("svm_clf", LinearSVC(C=20, loss="hinge", random_state=42, max_iter=3000))
    ])

polynomial_svm_clf.fit(X_whole, y_whole)
y_valid_pred = polynomial_svm_clf.predict(X_valid)
from sklearn.metrics import confusion_matrix
confusion_matrix_valid = confusion_matrix(y_valid, y_valid_pred)
print(confusion_matrix_valid) 

[[239  57  20]
 [ 70  22  17]
 [ 23  19  27]]


/home/ayush/Downloads/conda/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [32]:
y_valid_pred = polynomial_svm_clf.predict(X_valid)
from sklearn.metrics import confusion_matrix
confusion_matrix_valid = confusion_matrix(y_valid, y_valid_pred)
print(confusion_matrix_valid) 

[[239  57  20]
 [ 70  22  17]
 [ 23  19  27]]


In [33]:
#df_res.to_csv (r'/home/ayush/Downloads/export_dataframe3.csv', index = False, header=True)


In [164]:
from sklearn.ensemble import VotingClassifier

param_xgboost = {}
param_xgboost['objective'] = 'multi:softmax'
param_xgboost['num_class'] = len(np.unique(y_whole)) + 1 
param_xgboost['max_depth'] = 5
param_xgboost['learning_rate'] = 0.2
param_xgboost['gamma'] = 0

param_randomforest = {}
param_randomforest['n_estimators'] = 200
param_randomforest['max_depth'] = 10

param_svm = {}
param_svm['probability'] = True
param_svm['class_weight'] = 'balanced'
param_svm['C'] = 0.01
param_svm['gamma'] = 'scale'
param_svm['kernel'] = 'rbf'

param_knn = {}
param_knn['n_neighbors'] = 8
param_knn['leaf_size'] = 15


xgb = XGBClassifier(**param_xgboost)
knn = KNeighborsClassifier(**param_knn)
svm = SVC(**param_svm)
randomforest = RandomForestClassifier(**param_randomforest)

svm2 = SVC(C=0.01, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.15,
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.000001, verbose=False)


svm3 =  SVC(C=0.01, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.12,
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
 
#voting_clf2 = VotingClassifier(estimators=[('knn', knn), ('svm', svm), ('xgb', xgboost), ('RF', randomforest), 
#                                         ('svm2', svm),('svm3', svm3)],    voting='soft')

#voting_clf2 = VotingClassifier(estimators=[('knn', knn), ('svm', svm), ('xgb', xgboost), ('RF', randomforest)],    voting='soft')

voting_clf_i = VotingClassifier( estimators=[('knn', knn), ('svm3', svm3),('xgb', xgboost), ('RF', randomforest)],voting='hard')    


In [107]:
voting_clf_i.fit(X_train, y_train)

VotingClassifier(estimators=[('knn',
                              KNeighborsClassifier(leaf_size=15,
                                                   n_neighbors=8)),
                             ('svm3',
                              SVC(C=0.01, class_weight='balanced', gamma=0.12,
                                  probability=True)),
                             ('xgb',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                                            gpu_id=-1, importance_type='gain',
                                            interaction_constraints='',
                                            learning_rat...,
                                            max_depth=5, min_child_weight=1,
                                            missing=n

In [165]:

probas

[array([[0.54094258, 0.24488127, 0.21417615],
        [0.49415826, 0.25836559, 0.24747615],
        [0.49464941, 0.25804494, 0.24730565],
        ...,
        [0.92093805, 0.07311444, 0.0059475 ],
        [0.92127893, 0.07314855, 0.00557252],
        [0.92090237, 0.07333216, 0.00576546]]),
 array([[9.9449569e-01, 5.1914421e-03, 3.1290698e-04],
        [9.5541298e-01, 4.2916078e-02, 1.6710047e-03],
        [9.7625071e-01, 2.2697404e-02, 1.0519231e-03],
        ...,
        [9.5015806e-01, 4.7988955e-02, 1.8529593e-03],
        [9.5368356e-01, 4.5197282e-02, 1.1191630e-03],
        [9.3249869e-01, 6.5683946e-02, 1.8173965e-03]], dtype=float32),
 array([[0.97426057, 0.024115  , 0.00162443],
        [0.93481028, 0.06046026, 0.00472946],
        [0.94621973, 0.05040783, 0.00337244],
        ...,
        [0.91930821, 0.07641921, 0.00427257],
        [0.92307736, 0.07214096, 0.00478168],
        [0.91865572, 0.07651274, 0.00483154]])]

In [166]:
#probas[0],probas[1],probas[2]

(array([[0.54094258, 0.24488127, 0.21417615],
        [0.49415826, 0.25836559, 0.24747615],
        [0.49464941, 0.25804494, 0.24730565],
        ...,
        [0.92093805, 0.07311444, 0.0059475 ],
        [0.92127893, 0.07314855, 0.00557252],
        [0.92090237, 0.07333216, 0.00576546]]),
 array([[9.9449569e-01, 5.1914421e-03, 3.1290698e-04],
        [9.5541298e-01, 4.2916078e-02, 1.6710047e-03],
        [9.7625071e-01, 2.2697404e-02, 1.0519231e-03],
        ...,
        [9.5015806e-01, 4.7988955e-02, 1.8529593e-03],
        [9.5368356e-01, 4.5197282e-02, 1.1191630e-03],
        [9.3249869e-01, 6.5683946e-02, 1.8173965e-03]], dtype=float32),
 array([[0.97426057, 0.024115  , 0.00162443],
        [0.93481028, 0.06046026, 0.00472946],
        [0.94621973, 0.05040783, 0.00337244],
        ...,
        [0.91930821, 0.07641921, 0.00427257],
        [0.92307736, 0.07214096, 0.00478168],
        [0.91865572, 0.07651274, 0.00483154]]))

In [216]:
a = 1
b = 1
c = 1
probas = [c.fit(X_whole, y_whole).predict_proba(X_whole) for c in ( svm3, xgboost, randomforest)]
X_whole_proba = np.concatenate((a*probas[0],b*probas[1],c*probas[2]), axis=1)

probas_valid = [c.predict_proba(X_valid) for c in (svm3, xgboost, randomforest)]
X_valid_proba= np.concatenate((a*probas_valid[0],b*probas_valid[1],c*probas_valid[2]), axis=1)

In [217]:
#X_whole_proba[7998]

In [218]:
#y_whole

In [219]:
#X_valid_proba[493]

In [220]:
#y_valid

In [221]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import LinearSVC


polynomial_svm_clf=RandomForestClassifier(**param_randomforest)

polynomial_svm_clf.fit(X_whole_proba, y_whole)
y_valid_pred = polynomial_svm_clf.predict(X_valid_proba)
from sklearn.metrics import confusion_matrix
confusion_matrix_valid = confusion_matrix(y_valid, y_valid_pred)
print(confusion_matrix_valid) 
print(accuracy_score(y_valid, y_valid_pred)) 

[[250  50  16]
 [ 69  23  17]
 [ 17  14  38]]
0.6295546558704453


In [222]:
#voting_clf2.get_params()


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import LinearSVC

param_svm = {}
param_svm['C'] = 10
param_svm['gamma'] = 5
param_svm['kernel'] = 'rbf'



polynomial_svm_clf=SVC(**param_svm)

polynomial_svm_clf.fit(X_whole_proba, y_whole)
y_valid_pred = polynomial_svm_clf.predict(X_valid_proba)
from sklearn.metrics import confusion_matrix
confusion_matrix_valid = confusion_matrix(y_valid, y_valid_pred)
print(confusion_matrix_valid) 
print(accuracy_score(y_valid, y_valid_pred)) 

[[250  48  18]
 [ 68  26  15]
 [ 20  13  36]]
0.631578947368421


In [223]:
#voting_clf2.get_params()


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import LinearSVC

param_svm = {}
param_svm['C'] = 10
param_svm['coef0'] = 100
param_svm['kernel'] = 'poly'
param_svm['degree'] = 3



polynomial_svm_clf=SVC(**param_svm)

polynomial_svm_clf.fit(X_whole_proba, y_whole)
y_valid_pred = polynomial_svm_clf.predict(X_valid_proba)
from sklearn.metrics import confusion_matrix
confusion_matrix_valid = confusion_matrix(y_valid, y_valid_pred)
print(confusion_matrix_valid) 
print(accuracy_score(y_valid, y_valid_pred)) 





[[252  48  16]
 [ 71  24  14]
 [ 21  11  37]]
0.6336032388663968


In [162]:
y_valid_pred

array([0, 0, 0, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 1, 2, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 2, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 2, 1, 0, 1, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 2, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 0, 1, 0, 2, 0, 1, 1, 1, 2, 0, 0,
       0, 1, 2, 2, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       2, 0, 2, 2, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 1, 0, 0,
       1, 1, 0, 2, 2, 1, 2, 2, 2, 0, 0, 2, 1, 0, 0,

In [163]:
accuracy_score(y_valid, y_valid_pred)

0.6396761133603239

In [118]:
from sklearn.model_selection import GridSearchCV 



param_test1 = {
  'C':(.1,1,10,100,200,500,1000),
  'gamma':(.01,.1,1,5,10,100)
}
gsearch1 = GridSearchCV(estimator = SVC(kernel='rbf'), 
param_grid = param_test1,scoring='accuracy', n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_whole_proba,y_whole)
gsearch1.cv_results_

/home/ayush/Downloads/conda/lib/python3.8/site-packages/sklearn/model_selection/_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


{'mean_fit_time': array([1.3522326 , 1.03302822, 0.79739456, 0.80114489, 0.97343154,
        2.26054082, 1.1617938 , 1.0247005 , 0.72209215, 0.72494769,
        0.82063231, 3.38435535, 0.97173648, 0.73378878, 0.66340919,
        0.726086  , 0.85953283, 3.97713647, 0.82042675, 0.689713  ,
        0.8419579 , 1.19219499, 1.35051517, 5.48957863, 0.82800398,
        0.727247  , 1.0627018 , 1.54070077, 1.67700119, 6.20185304,
        0.82678051, 0.81227884, 1.43656716, 2.3506218 , 2.3803648 ,
        7.20602689, 0.79308858, 0.95202498, 1.9699194 , 3.1234622 ,
        3.38711987, 8.78985286]),
 'std_fit_time': array([0.01505104, 0.03417304, 0.0614999 , 0.0237097 , 0.03856123,
        0.13582422, 0.0322265 , 0.14689771, 0.03225455, 0.01505541,
        0.02583369, 0.183741  , 0.07318979, 0.03551459, 0.02799299,
        0.01037341, 0.01741958, 0.46332475, 0.05764495, 0.03008568,
        0.03796941, 0.06985289, 0.02965081, 0.64426102, 0.04017501,
        0.02775698, 0.08076243, 0.04065939, 0.168

In [123]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

poly_features = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly_features.fit_transform(X_whole_proba)

lin_reg = LinearRegression()
lin_reg.fit(X_poly, y_whole)


LinearRegression()

In [124]:
X_poly_valid = poly_features.fit_transform(X_valid_proba)

In [125]:
print(lin_reg.score(X_poly_valid, y_valid))

0.20424125481745936
